In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".80"

import numpy as np

import jax
import jax.numpy as jnp
import spyx
import haiku as hk
import optax
from jax_tqdm import scan_tqdm

### SHD Dataloading

In [2]:

import tonic
from tonic import datasets, transforms
import torchvision as tv
from torch.utils.data import DataLoader, Subset
from collections import namedtuple

State = namedtuple("State", "obs labels")



In [3]:
class _SHD2Raster():
    """ 
    Tool for rastering SHD samples into frames. Packs bits along the temporal axis for memory efficiency. This means
        that the used will have to apply jnp.unpackbits(events, axis=<time axis>) prior to feeding the data to the network.
    """

    def __init__(self, encoding_dim, sample_T = 100):
        self.encoding_dim = encoding_dim
        self.sample_T = sample_T
        
    def __call__(self, events):
        # tensor has dimensions (time_steps, encoding_dim)
        tensor = np.zeros((events["t"].max()+1, self.encoding_dim), dtype=int)
        np.add.at(tensor, (events["t"], events["x"]), 1)
        #return tensor[:self.sample_T,:]
        tensor = tensor[:self.sample_T,:]
        tensor = np.minimum(tensor, 1)
        tensor = np.packbits(tensor, axis=0)
        return tensor

In [4]:
sample_T = 64
shd_timestep = 1e-6
shd_channels = 700
net_channels = 128
net_dt = 1/sample_T
batch_size = 256

obs_shape = tuple([net_channels,])
act_shape = tuple([20,])

transform = transforms.Compose([
    transforms.Downsample(
        time_factor=shd_timestep / net_dt,
        spatial_factor=net_channels / shd_channels
    ),
    _SHD2Raster(net_channels, sample_T=sample_T)
])

train_dataset = datasets.SHD("./data", train=True, transform=transform)
test_dataset = datasets.SHD("./data", train=False, transform=transform)



In [5]:
train_dl = iter(DataLoader(train_dataset, batch_size=len(train_dataset),
                          collate_fn=tonic.collation.PadTensors(batch_first=True), drop_last=True, shuffle=False))
        
x_train, y_train = next(train_dl)

In [6]:
test_dl = iter(DataLoader(test_dataset, batch_size=len(test_dataset),
                          collate_fn=tonic.collation.PadTensors(batch_first=True), drop_last=True, shuffle=False))
        
x_test, y_test = next(test_dl)

In [7]:
print(type(x_train))
print(x_train.shape)

<class 'torch.Tensor'>
torch.Size([8156, 8, 128])


In [8]:
x_train = jnp.array(x_train, dtype=jnp.uint8)
y_train = jnp.array(y_train, dtype=jnp.uint8)

In [10]:
def shuffle(dataset, shuffle_rng):
    x, y = dataset

    cutoff = y.shape[0] % batch_size

    obs = jax.random.permutation(shuffle_rng, x, axis=0)[:-cutoff]
    labels = jax.random.permutation(shuffle_rng, y, axis=0)[:-cutoff]

    obs = jnp.reshape(obs, (-1, batch_size) + obs.shape[1:])
    labels = jnp.reshape(labels, (-1, batch_size)) # should make batch size a global

    return State(obs=obs, labels=labels)

In [11]:

key = jax.random.PRNGKey(0)
x, y = x_train, y_train


### Spyx SHD

In [19]:
def shd_snn(x): # need to constrain beta to be 0.8...
        
    core = hk.DeepRNN([
        hk.Linear(64, with_bias=False),
        spyx.nn.LIF((64,), beta=0.8, activation=spyx.axn.Axon(spyx.axn.arctan())),
        hk.Linear(64, with_bias=False),
        spyx.nn.LIF((64,), beta=0.8, activation=spyx.axn.Axon(spyx.axn.arctan())),
        hk.Linear(20, with_bias=False),
        spyx.nn.LI((20,), beta=0.8)
    ])
    
    # static unroll for maximum performance
    spikes, V = hk.dynamic_unroll(core, x, core.initial_state(x.shape[0]), time_major=False, unroll=32)
    
    return spikes, V

In [20]:
key = jax.random.PRNGKey(0)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
sample_x, sample_y = shuffle((x,y),key)
SNN = hk.without_apply_rng(hk.transform(shd_snn))
params = SNN.init(rng=key, x=jnp.float16(sample_x[0]))

In [21]:
def gd(SNN, params, dataset, epochs=300):
        
    opt = optax.adam(learning_rate=5e-4)
    
    # create and initialize the optimizer
    opt_state = opt.init(params)
    grad_params = params
        
    # define and compile our eval function that computes the loss for our SNN
    @jax.jit
    def net_eval(weights, events, targets):
        readout = SNN.apply(weights, events)
        traces, V_f = readout
        return spyx.fn.integral_crossentropy(traces, targets) # smoothing needs to be more explicit in docs...
        
    # Use JAX to create a function that calculates the loss and the gradient!
    surrogate_grad = jax.value_and_grad(net_eval) 
        
    rng = jax.random.PRNGKey(0)        
    
    # compile the meat of our training loop for speed
    @jax.jit
    def train_step(state, data):
        grad_params, opt_state = state
        events, targets = data # fix this
        events = jnp.unpackbits(events, axis=1) # decompress temporal axis
        # compute loss and gradient                    # need better augment rng
        loss, grads = surrogate_grad(grad_params, events, targets)
        # generate updates based on the gradients and optimizer
        updates, opt_state = opt.update(grads, opt_state, grad_params)
        # return the updated parameters
        new_state = [optax.apply_updates(grad_params, updates), opt_state]
        return new_state, loss
    
    
    # Here's the start of our training loop!
    @scan_tqdm(epochs)
    def epoch(epoch_state, epoch_num):
        curr_params, curr_opt_state = epoch_state

        shuffle_rng = jax.random.fold_in(rng, epoch_num)
        train_data = shuffle(dataset, shuffle_rng)
        
        # train epoch
        end_state, train_loss = jax.lax.scan(
            train_step,# func
            [curr_params, curr_opt_state],# init
            train_data,# xs
            train_data.obs.shape[0]# len
        )
                    
        return end_state, jnp.mean(train_loss)
    # end epoch
    
    # epoch loop
    final_state, metrics = jax.lax.scan(
        epoch,
        [grad_params, opt_state], # metric arrays
        jnp.arange(epochs), # 
        epochs # len of loop
    )
    
    final_params, _ = final_state
    
                
    # return our final, optimized network.       
    return final_params, metrics

In [22]:
def test_gd(SNN, params, dataset):

    @jax.jit
    def test_step(params, data):
        events, targets = data
        events = jnp.unpackbits(events, axis=1)
        readout = SNN.apply(params, events)
        traces, V_f = readout
        acc, pred = spyx.fn.integral_accuracy(traces, targets)
        loss = spyx.fn.integral_crossentropy(traces, targets)
        return params, [acc, loss, pred, targets]
    
    test_data = shuffle(dataset, jax.random.PRNGKey(0))
    
    _, test_metrics = jax.lax.scan(
            test_step,# func
            params,# init
            test_data,# xs
            test_data.obs.shape[0]# len
    )
    
    acc = jnp.mean(test_metrics[0])
    loss = jnp.mean(test_metrics[1])
    preds = jnp.array(test_metrics[2]).flatten()
    tgts = jnp.array(test_metrics[3]).flatten()
    return acc, loss, preds, tgts

In [29]:
grad_params, metrics = gd(SNN, params, (x,y), epochs=300)


  0%|          | 0/300 [00:00<?, ?it/s]

In [30]:
metrics

Array([9.956848 , 3.3741896, 3.0981524, 3.098117 , 3.10788  , 3.0800176,
       3.044519 , 3.0078552, 2.9578753, 2.9254699, 2.9000227, 2.855232 ,
       2.825492 , 2.8034348, 2.7728598, 2.7524285, 2.7449524, 2.7288525,
       2.6960046, 2.6906924, 2.6746328, 2.656041 , 2.648154 , 2.628879 ,
       2.6072752, 2.59138  , 2.5787544, 2.55287  , 2.537342 , 2.5289085,
       2.5115821, 2.492704 , 2.485445 , 2.4841418, 2.4817295, 2.461743 ,
       2.4588144, 2.4411714, 2.4179587, 2.4100943, 2.4141183, 2.4042754,
       2.3774354, 2.371575 , 2.3634982, 2.3623655, 2.3439069, 2.3433695,
       2.3426375, 2.3337328, 2.326334 , 2.3270352, 2.3221889, 2.3201895,
       2.31026  , 2.3067636, 2.2986856, 2.2941604, 2.2861533, 2.2901187,
       2.2763484, 2.2621   , 2.2566044, 2.2695832, 2.2541065, 2.2461262,
       2.249047 , 2.245604 , 2.2367456, 2.2299867, 2.2237453, 2.2228508,
       2.2209733, 2.2107854, 2.2181134, 2.2184722, 2.2136185, 2.204432 ,
       2.1955223, 2.207146 , 2.200374 , 2.1876988, 

In [31]:
acc, loss, preds, tgts = test_gd(SNN, grad_params, (x,y))
print("Accuracy:", acc, "Loss:", loss)

Accuracy: 0.81413805 Loss: 1.9373717
